In [207]:
import os
import requests
import pandas as pd
from lxml import etree
from bs4 import BeautifulSoup

In [208]:
d1 = pd.read_csv('disease_links.csv')

# base_url = "https://rarediseases.info.nih.gov/"
links = []

for i in range(5910):
    sub_url = d1.loc[i, "link"].split('/')[-1]
    links.append(sub_url)
print(links[:100])

['10q223q23-microdeletion-syndrome', '11-beta-hydroxylase-deficiency', '12q14-microdeletion-syndrome', '15q112-microdeletion', '15q133-microdeletion-syndrome', '15q133-microduplication-syndrome', '15q24-microdeletion-syndrome', '16p112-deletion-syndrome', '16p112-duplication', '16p1311-microduplication-syndrome', '16q243-microdeletion-syndrome', '17-alpha-hydroxylase-deficiency', '17-beta-hydroxysteroid-dehydrogenase-3-deficiency', '17q12-deletion-syndrome', '17q12-duplication', '17q231q232-microdeletion-syndrome', '18-hydroxylase-deficiency', '19p1312-microdeletion-syndrome', '1q-duplications', '1q211-microdeletion-syndrome', '1q44-microdeletion-syndrome', '24-dienoyl-coa-reductase-deficiency', '2-hydroxyethyl-methacrylate-sensitization', '2-hydroxyglutaric-aciduria', '2-methylacetoacetyl-coa-thiolase-deficiency', '2-methylbutyryl-coa-dehydrogenase-deficiency', '20p123-microdeletion-syndrome', '21-hydroxylase-deficiency', '22q112-deletion-syndrome', '22q112-duplication-syndrome', '22q

In [209]:
viol = 0
for link in links:
    file_path = os.path.join('pages', link)
    if not os.path.exists(file_path):
        print(link)
        viol += 1
if viol > 0:
    print(f"Total {viol} violations found. Check and get back.", end=" ")
else:
    print("Looks good to me, proceed...", end=" ")
print(f'[{len(links) - viol}/{len(links)}] working')

Looks good to me, proceed... [5910/5910] working


In [210]:
data = []
df = pd.DataFrame(data)

In [211]:
num = 0
for link in links:
    file_path = os.path.join('pages', link)
    with open(file_path, "r") as file:
        html = file.read()

    soup = BeautifulSoup(html, "html.parser")
    d = {}
    glance = soup.find('app-disease-at-a-glance-summary').div.text
    if 'This section is currently in development.' in glance:
        glance = ''
    d['glance'] = glance
    affected = soup.find('app-disease-at-a-glance-people-affected').div.text.replace('than', 'than ')
    if 'This section is currently in development.' in affected:
        affected = ''
    d['affected'] = affected
    symptoms = soup.find('app-symptoms-section')
    symptomlist = []
    symptoms_s = symptoms.find('div', class_='symptom-list-container')
    if symptoms_s is not None:
        symptoms_s = symptoms_s.find_all('button')
        for symptom in symptoms_s:
            symptomlist.append(symptom.text)
    # print(symptomlist)
    d['symptoms'] = '; '.join(symptomlist)
    categories = soup.find('div', class_='categories-body')
    d['categories'] = '; '.join(categories.text.split(';  '))
    common_ages = soup.find('div', class_='ms-xl-5')
    agelist = []
    if common_ages is not None:
        common_ages = common_ages.find_all('div', class_="active")
        for i in common_ages:
            if i is not None:
                spantext = i.find('span')
                if spantext is not None:
                    agelist.append(i.find('div', class_="under-title").text.replace(spantext.text, ''))
    d['ages'] = '; '.join(agelist)
    causes = soup.find('app-causes-container')
    causelist_s = causes.find_all('h5')
    causelist = []
    for i in causelist_s:
        causelist.append(i.text)
    d['causes'] = '; '.join(causelist)
    dict = pd.DataFrame(d, index=[0])
    df = pd.concat([df, dict], ignore_index=True)
df.to_csv('disease_details.csv', index=False)

In [204]:
file_path = 'pages/occipital-horn-syndrome'
with open(file_path, "r") as file:
    html = file.read()

soup = BeautifulSoup(html, "html.parser").body
# print(soup)

In [205]:
glance = soup.find('app-disease-at-a-glance-summary').div.text
if 'This section is currently in development.' in glance:
    glance = ''
df['glance'] = glance
affected = soup.find('app-disease-at-a-glance-people-affected').div.text.replace('than', 'than ')
if 'This section is currently in development.' in affected:
    affected = ''
df['affected'] = affected

In [206]:
symptoms = soup.find('app-symptoms-section')
symptomlist = []
symptoms_s = symptoms.find('div', class_='symptom-list-container')
if symptoms_s is not None:
    symptoms_s = symptoms_s.find_all('button')
    for symptom in symptoms_s:
        symptomlist.append(symptom.text)
print(symptomlist)
df['symptoms'] = '; '.join(symptomlist)

[' Abnormal skull morphology ', ' Abnormality of the face ', ' Cerebral calcification ', ' Delayed cranial suture closure ', ' Exostoses ', ' Global developmental delay ', ' Hyperextensible skin ', ' Intellectual disability ', ' Joint hyperflexibility ', ' Large fontanelles ', ' Specific learning disability ', ' Abnormal esophagus physiology ', ' Abnormality of the sense of smell ', ' Abnormality of the wrist ', ' Atypical scarring of skin ', ' Brachydactyly ', ' Bruising susceptibility ', ' Cholestasis ', ' Dysphagia ', ' Esophagitis ', ' Gastroesophageal reflux ', ' Gastroparesis ', ' Hepatitis ', ' Hiatus hernia ', ' High, narrow palate ', ' Hypotonia ', ' Jaundice ', ' Keloids; Keloid scar ', ' Long philtrum ', ' Osteomalacia ', ' Osteopenia ', ' Osteoporosis ', ' Pectus carinatum ', ' Pectus excavatum ', ' Platyspondyly ', ' Poor suck ', ' Rickets ', ' Scarring ', ' Short palm ', ' Synostosis of joints ', ' Vascular dilatation ', ' Venous insufficiency ', ' Abnormality of fibula m

In [186]:
categories = soup.find('div', class_='categories-body')
# print('; '.join(categories.text.split(';  ')))
df['categories'] = '; '.join(categories.text.split(';  '))

In [187]:
common_ages = soup.find('div', class_='ms-xl-5')
agelist = []
if common_ages is not None:
    common_ages = common_ages.find_all('div', class_="active")
    for i in common_ages:
        if i is not None:
            spantext = i.find('span')
            if spantext is not None:
                agelist.append(i.find('div', class_="under-title").text.replace(spantext.text, ''))
df['ages'] = '; '.join(agelist)

In [189]:
causes = soup.find('app-causes-container')
causelist_s = causes.find_all('h5')
causelist = []
for i in causelist_s:
    causelist.append(i.text)
df['causes'] = '; '.join(causelist)

[]
